In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       
        
        url_main = f"https://www.sportdoma.ru/catalog/trenazher/{category}/{page}.html?sort=populartop&type=all&marka=all&seria=all&all="
        response = requests.get(url_main, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'name')
        for item in data:
                if item.find('a') != None:
                    if bool(re.search(pattern = re.escape('catalog'), string = item.find('a').attrs['href'])):
                        item_url = 'https://www.sportdoma.ru' + item.find('a').attrs['href']
                        yield item_url


In [3]:
def get_item(category, tot_pages):
    
    for url_item in get_url(category, tot_pages):
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        
        try:
            title = soup.find('h1').text
        except:
            title = ''

        try:
            article = soup.find(class_ = 'art').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '')
        except:
            article = ''

        try:
            image = soup.find(class_ = 'big_foto')
            url = 'https://www.sportdoma.ru' + image.find('a').attrs['href']

        except:
            print(f'Log: no image for {article} found...')
            url = ''
   
        try:
            info_lst = []
            info = soup.find_all(class_ = 'text_block3')
            for item in info:
                text = item.text.replace('\n', '').strip()
                text = re.sub(r"(?<=\w)([А-Я])", r" \1", text)
                info_lst.append(text.strip())

            desc = info_lst[1]
            char = info_lst[0]
        except:
            char = ''
            desc = ''

        try:
            price = soup.find(class_ = 'number').text
            price = price.replace('Р.', '').replace(' ', '')

        except:
            price = ''

        try:
            cat = soup.find(id = 'breadcramps').text
            temp_str = re.sub(re.compile('[^а-яА-Я ]'), '', cat)
            lst = re.findall('[А-Я][^А-Я]*', temp_str)

            try:
                cat_1 = lst[-2]
            except:
                cat_1 = ''
            try:
                cat_2 = lst[-1]
            except:
                cat_2 = ''
            try:
                cat_3 = re.sub(re.compile('[^a-zA-Z ]'), '', cat).strip()
            except:
                cat_3 = ''
        except:
            cat_1 = ''
            cat_2 = ''
            cat_3 = ''
    
        yield title, article, price, cat_1, cat_2, cat_3, url, desc, char

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics']

df = pd.DataFrame(columns = df_columns)
save_frequency = 200
log_frequency = 5

In [5]:
def to_csv(category, tot_pages):
    counter = 0  

    for item in get_item(category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            #time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv('sportdoma.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv('sportdoma.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('stepper', 3)


Total items saved: 98


In [9]:
to_csv('begovaya-dorozhka', 7)

Log: working over page 5...
200 items saved in csv..

Total items saved: 301


In [10]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=200></img></a>'.format(val, val)
display(df.style.format({'url': show_image, **{'width': '100px'}})) 

,title,article,price,cat_1,cat_2,cat_3,url,description,caracteristics
0,Степпер DFC SС-S039,14819,21990,Степперы,Гидравлические степперы,DFC,,Домашний степпер DFC SC-S039 со стойкой и удобным поручнем. Измерение пульса при помощи датчиков на поручне. Встроенные колеса для перемещения. Держатель бутылки. Гидроцилиндр сопротивления настраивается за 12 шагов.,"Тип степпера:домашний Система нагружения:2 гидроцилиндра Регулировка нагрузки:нет Показания консоли:время, количество шагов за тренировку, ритм, шаг/мин, израсходованные калории Многоязычный интерфейс:нет Питание:не требует Складывание:нет Габариты (Дх Шх В):62х82х134 см, в коробке: 82х50х21 см Вес:22 кг, в коробке: 24 кг Максимальный вес пользователя:120 кг Гарантия:1 год Производитель:DFC Страна изготовления:Китай"
1,Степпер SOLE SC200 (CC81 2019) Cardio Climber,19863,129000,Степперы,Электромагнитные степперы,SOLE,,"Тренажер объединяет два упражнения в одном, классический степпер и тренажер climber. Тренировка обеспечивает комплексную кардио-силовую тренировку вместе с интенсивным сжиганием калорий, и при этом тренажер полностью безопасен для суставов и спины. Sole CC81 Cardio Climber уникальный тренажер на рынке, который превосходит по эффективности все классические кардиотренажеры. Инженеры Sole Fitness продумали все детали для безопасных, эффективных и комфортных тренировок. Тренажер CC81 сочетает в себе все необходимые движения для развития мышц участвующих в скалолазании, которое является очень энергозатратным и обеспечивает комплексную тренировку мышц всего тела: мышц рук, ног, туловища, спины, ягодичных мышц и квадрицепсов, а также мышц пресса. Диск сопротивления на консоли позволяет пользователям переключаться между интервалами высокой и низкой интенсивности (отдыха), что обеспечивает эффективную интервальную тренировку. Легко регулируемые рукоятки с 4 настройками по высоте дают пользователям разного роста возможность подобрать комфортный ход рук и плечевого пояса. Предусмотрены разные хваты, чтобы включить в работу трицепс и бицепс или переключить фокус на дельтовидные мышцы.","Тип степпера:для домашнего использования Система нагружения:электромагнитная ECB Маховик:12 кг.Измерение пульса:кардиопояс (в комплекте)Консоль:5” x 3” LCD Кол-во программ:ручная программа, 6 предустановленных программ, 2 пользовательских программы, 2 кардиопрограммы Интеграция и Мультимедиа:Bluetooth / Мобильное приложение: Sole App, Apple Health / MyFitnessPal, аудиосистема MP3 Питание:сеть 220 Вольт Складывание:нет Габариты (Дх Шх В):в рабочем состоянии 150 х 79 х 219 см.; в упаковке 150 х 56 х 102 см.Вес:нетто 120 кг.Максимальный вес пользователя:181 кг.Гарантия:2 года Производитель:DYACO International INC, С Ш А Страна изготовления:Тайвань"
2,Степпер DFC SC-S085G Поворотный с поручнем,20079,8890,Степперы,Министепперы,DFC,,"Поворотный мини-степпер c поручнями прорабатывает и тонизирует ягодицы и мышцы ног, снижая давление на кости и суставы. Система нагружения — 2 гидравлических цилиндра. Взаимозависимый ход педалей. Возможность изменения нагрузки и регулировка высоты педалей. Поручни для дополнительной устойчивости пользователя, регулируются по углу наклона. Нескользящая поверхность педалей (платформ) для ног.","Тип степпера:для домашнего использования Система нагружения:2 гидравлических цилиндра Педали:нескользящие, взаимозависимые с регулировкой нагрузки и высоты Консоль:монохромный LCD Показания консоли:время, количество шагов за тренировку, ритм (количество шагов в минуту), калории, скан.Питание:консоли - батарейки Габариты (Дх Шх В):в рабочем состоянии 46 х 40,5 х 138 см.; в упаковке 45 х 24 х 51 см.Вес:нетто 9.5 кг.; брутто 10.2 кг.Максимальный вес пользователя:до 100 кг Гарантия:12 мес.Производитель:DFC Страна изготовления:Китай"
3,Степпер DFC SC-S083G,20080,5890,Степперы,Министепперы,DFC,,Мини-степпер прорабатывает и тонизирует ягодицы и мышцы ног. Система нагружения — 2 гидравлических цилиндра. Взаимозависимый ход педалей. Нескользящая поверхность педалей (платформ) для ног.,